# Learning To use the Bible

## Using Holistic on an image

In [9]:
from sign.training.landmark_extraction.HolisticPiper import HolisticPiper
from pathlib import Path

FRAMES_PATH = "../backend/dynamic_signs/frames"
OUT_PATH = Path().cwd().absolute().joinpath("csvs")
OUT_PATH_STR = str(OUT_PATH.absolute())
if not OUT_PATH.exists():
    OUT_PATH.mkdir()

piper = HolisticPiper(out_dest=OUT_PATH)

piper.write_dynamic_gestures_from_folder_to_csv(FRAMES_PATH, "", "")
piper.out_dest

2024-04-06 15:55:57.293010: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-06 15:55:57.293443: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-06 15:55:57.295864: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-06 15:55:57.328423: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-06 15:55:58.265199: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

PosixPath('/home/chobbie/Desktop/bach/Bsc-Sign-Language-Recon/notebooks/csvs')

In [10]:
from sign.training.load_data.HolisticCsvReader import HoslisticCsvReader

reader = HoslisticCsvReader()
result = reader.extract_holistic_landmarks_from_folder(OUT_PATH_STR)
f"Parsed {len(result)} classes"
#first_class = list(result.keys())[1]
#len(result[first_class]), first_class

'Parsed 2 classes'

In [ ]:
## BORROW FROM DYNAMIC GESTURE
from random import shuffle
import numpy as np
from typing import Iterable, Tuple, TypeVar
from sign.trajectory import TrajectoryBuilder
from sign.landmarks import NormalizedLandmark, pre_process_landmark, calc_landmark_list
from dynamic_signs.csv_reader import csv_reader
from sklearn.model_selection import train_test_split
bob = TrajectoryBuilder(target_len=24)


def shuffle_training_data(data: Iterable, labels: Iterable) -> Tuple[Iterable,Iterable]:
    zipped = list(zip(data, labels))
    shuffle(zipped)
    return tuple(zip(*zipped))
    

def extract_training_data_and_labels_from_dynamic_gesture_map(gesture_map: dict[str, list[np.ndarray]]) -> Tuple[Iterable[np.ndarray], Iterable[str]]:
    trajectories_and_landmarks: list[np.ndarray] = []
    labels : list[str] = []
    for label, label_data in gesture_map.items():
        for data in label_data:
            labels.append(label)
            trajectories_and_landmarks.append(data)
    return shuffle_training_data(trajectories_and_landmarks, labels)

def prune_training_data_and_labels_from_dynamic_gesture_csv(input: dict[str, dict[int, list[float]]]) -> dict[str, list[np.ndarray]]:
    target_length = 24*3*21
    bob = TrajectoryBuilder(target_len=target_length)
    res: dict[str, list[np.ndarray]] = {}
    for label, videos in input.items():
        for id, frames in videos.items():
            if len(frames) == 0:
                print(f"{label}-{id} is empty - SKIPPING")
                continue
            existing = res.get(label)
            if len(frames) < target_length:
                frames = bob.pad_sequences_of_landmarks(frames)
            else: 
                frames = bob.extract_keyframes_sample(frames)
            frames = np.array(frames)
            if existing is not None:
                existing.append(frames)
            else:
                res[label] = [frames]
                
    return res

def extract_left_hand_landmarks_from_holistic(dict: dict[str, dict[int, HolisticSequence]]) -> dict[str, dict[int, list[float]]]:
    res = {}
    for label, video_id_to_holistic_seq in dict.items():
        res[label] = {} 
        for id, holy_seq in video_id_to_holistic_seq.items():
            if len(holy_seq["left_hand"]) > 0:
                res[label][id] = holy_seq["left_hand"]
            elif len(holy_seq["right_hand"]) > 0:
                res[label][id] = holy_seq["right_hand"]
            #else --> Do nothing!!??
            
    return res

def extract_training_data_and_labels_from_dynamic_gesture_csv() -> Tuple[Iterable[np.ndarray], Iterable[str]]:
    unpruned = reader.extract_holistic_landmarks_from_folder(OUT_PATH_STR)
    unpruned = extract_left_hand_landmarks_from_holistic(unpruned)
    pruned = prune_training_data_and_labels_from_dynamic_gesture_csv(unpruned)
    normalized_landmarks = {}
    
    for key, val in pruned.items():
        for seq in val:
            trajectory = bob.make_trajectory(seq.reshape(-1, 21, 3))
            landmarks = []
            for i in range(0, len(seq), 3):
                landmark = NormalizedLandmark()
                landmark.x = seq[i]
                landmark.y = seq[i+1]
                landmark.z = seq[i+2]
                landmarks.append(landmark)
            normalized_landmarks_for_video = pre_process_landmark(calc_landmark_list(landmarks))
            existing = normalized_landmarks.get(key)
            asd = trajectory.to_float_list()
            asd.extend(normalized_landmarks_for_video)
            with_trajectories = np.array(asd)
            
            if existing is None:
                normalized_landmarks[key] = [with_trajectories]
            else:
                existing.append(with_trajectories)
    return extract_training_data_and_labels_from_dynamic_gesture_map(normalized_landmarks)

extracted_data,extracted_labels = extract_training_data_and_labels_from_dynamic_gesture_csv()

shapes = list(map(lambda arr : arr.shape ,extracted_data))
first = extracted_data[0].shape
f"Shape of first {first}. Do all data have this shape? '{'Yes' if all(first == s for s in shapes) else 'No'}'"

In [ ]:
#Holistic on one-armed-bandit looking for hands- Holy nation approves of this one 
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

labels_J = (np.array(extracted_labels, dtype=np.str_) == "J")
model_J_or_not = make_pipeline(StandardScaler(),
                          SVC(kernel="poly", degree=6, coef0=1))
model_J_or_not.fit(extracted_data, labels_J)

if False:
    from joblib import dump
    dump(model_svm, 'dynamic_model.joblib')
model_J_or_not

In [ ]:
# ABSOLUTE WORST TEST, please correct me :3
toPredict = [extracted_data[0]]
true_label = labels_J[0]
print(f"Is J?\n  Predicted: {model_J_or_not.predict(toPredict)[0]}\n  Should be: {true_label}")